In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [22]:
import tarfile
import numpy as np 
import pandas as pd
import cv2
import string
import os
from sklearn.metrics import accuracy_score
from skimage.feature import hog
from PIL import Image
from google.colab.patches import cv2_imshow
from sklearn.metrics import classification_report

In [3]:
tar = tarfile.open("/content/gdrive/My Drive/Lab0609.tar.gz", "r:gz")
tar.extractall('./hand_gestures')
tar.close()

In [4]:
os.chdir("hand_gestures/Lab0609") #run this only once 

In [5]:
os.listdir()

['B', 'G', 'F', 'I', 'nothing', 'E', 'D', 'H', 'C', 'A']

## Features given in the description will be used:

1.  [ Harris Corner Detection](https://docs.opencv.org/3.4/dc/d0d/tutorial_py_features_harris.html): It is a grayscale image with scores across the corner of an image.
2.   [Histogram of Gradients](https://www.learnopencv.com/histogram-of-oriented-gradients/): The technique counts occurrences of gradient orientation in localized portions of an image
3.   Number of white pixels: Number of white pixels in a binary b-w image.
4.   [ORB](https://docs.opencv.org/3.4/d1/d89/tutorial_py_orb.html): Oriented FAST and Rotated BRIEF: Opensource and faster feature descriptor that gives 128 main features.
5.   Statistical Features from multi-dimensions: Inference of features through transforation operation on images.





### 1. Converting RGB to binary image and train test split
### 2. Storing image features in np.array ***data***




In [6]:
from os import listdir
#feature extraction done below
def load_doc(filename):
    file = open(filename)
    imga = cv2.imread(filename, 2)
    img = cv2.resize(imga, (20, 20))
    #binary image and features 
    im_bw = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]
    white_pixel = np.count_nonzero(im_bw >= 250)
    #hog_features, hog_image = hog(img,visualize=True,block_norm='L2-Hys',pixels_per_cell=(5, 5))
    topographical_features = im_bw.flatten()
    imgcd = cv2.resize(imga, (10, 10))
    im_cd = cv2.threshold(imgcd, 127, 255, cv2.THRESH_BINARY)[1] 
    hcd = cv2.cornerHarris(imgcd,2,3,0.04)
    orb = cv2.ORB_create()
    kp = orb.detect(img,None)
    kp, des = orb.compute(img, kp)
    grd = np.array(im_cd.flatten())
    fin = np.hstack((topographical_features,white_pixel))
    file.close()
    return [fin, grd]
      
def process_reviews(directory):
  v = []
  b = []
  for filename in listdir(directory):
    a = load_doc('/content/hand_gestures/Lab0609/'+directory + '/' + filename) 
    v.append(a[0])
    b.append(a[1])
  return [v,b]

def train_test_split(data, labels):
  X_train = []
  y_train = []
  X_test = []
  y_test = []
  for i in range(0,10):
    for j in range(0,2100):
      X_train.append(data[i][j])
      y_train.append(labels[i][j])
    for j in range(2100,3000):
      X_test.append(data[i][j])
      y_test.append(labels[i][j])
  return X_train, y_train, X_test, y_test

In [7]:
#DATA CONTAINES FEATURE VECTOR FOR SVM CLASSIFICATION 
img = []
data = []
Y = []
gridf = []
for dir in os.listdir():
  img = process_reviews(dir)
  data.append(img[0])
  gridf.append(img[1])
  Y.append(dir)
  img = []

print(len(data))
print(len(data[0]))

10
3000


In [10]:
#We take note of corresponding file name based on their postion in array and store them into a csv
file_name = []
for dir in os.listdir():
  for f in listdir(dir):
    file_name.append(f)

In [11]:
labels = []
label = os.listdir()
for i in range(0,10):
  key = label[i]
  lab = []
  for j in range(0,3000):
    lab.append(key)
  labels.append(lab)

In [12]:
# CSV file based on image features and labels saved as data.csv
df = pd.DataFrame(data[0]) 
df['labels'] = labels[0]

for i in range(1,10):
  df1 = pd.DataFrame(data[i])
  df1['labels'] = labels[i]
  df = pd.concat([df,df1],ignore_index=True)
df['filename'] = file_name
#PLEASE UNCOMMENT THE LINE BELOW TO GET CSV FOR THE DATASET
#df.to_csv('/content/data.csv') 

##### Please uncomment the df.to_csv *('/content/data.csv')* part to get a csv file with features, the notebook directly uses the feature array for training

In [13]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,labels,filename
0,0,0,0,0,0,255,255,255,255,255,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,255,255,0,0,0,0,0,0,...,255,255,255,255,255,255,255,255,255,255,0,0,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,0,0,255,255,255,255,255,307,B,B1474.jpg
1,0,0,0,0,0,255,255,255,255,255,255,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,255,255,0,0,0,0,0,0,...,255,255,255,255,255,0,0,0,0,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,0,0,0,0,255,255,255,255,255,255,255,255,257,B,B1002.jpg
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,255,0,0,0,0,0,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,0,0,0,0,0,255,255,255,255,255,255,255,255,255,255,255,139,B,B2036.jpg
3,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,255,255,0,0,0,0,255,255,255,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,255,0,0,0,0,0,255,255,255,255,0,0,0,0,0,0,0,0,255,255,255,0,0,0,0,0,255,255,255,255,187,B,B2589.jpg
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,255,255,255,255,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,0,0,0,0,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,0,0,0,0,0,255,255,255,255,255,255,255,238,B,B761.jpg


In [14]:
X_train, y_train, X_test, y_test = train_test_split(data, labels)
X_train2, y_train2, X_test2, y_test2 = train_test_split(gridf, labels)

###  3. Use the features for classification using SVM (default setting) and classification report

In [15]:
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline 
model = make_pipeline(svm.SVC())
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [16]:
y_pred = model.predict(X_train)

In [17]:
#Classification Report for default SVM classifier
print(classification_report(y_train, y_pred, target_names=label))

              precision    recall  f1-score   support

           B       0.97      0.94      0.96      2100
           G       0.92      0.92      0.92      2100
           F       0.97      0.97      0.97      2100
           I       0.94      0.97      0.95      2100
     nothing       0.94      0.93      0.93      2100
           E       0.94      0.95      0.94      2100
           D       0.98      0.98      0.98      2100
           H       0.98      0.98      0.98      2100
           C       0.99      0.99      0.99      2100
           A       1.00      1.00      1.00      2100

    accuracy                           0.96     21000
   macro avg       0.96      0.96      0.96     21000
weighted avg       0.96      0.96      0.96     21000



### 4. Appling GridSearch for hyperparameter optimization on the train feature set and 

In [18]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(kernel = 'rbf'), {"C": [1,5,10],'gamma':[0.1,5],}, cv=3, return_train_score=False,n_jobs=-1)
clf.fit(X_train2, y_train2)
clf.cv_results_
print(clf.best_params_)
#Kernal examination at default setting for linear and rbf gives better result using RBF abd could be understood because of non-linear functionality

{'C': 1, 'gamma': 0.1}


In [19]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,57.344575,0.090604,17.068243,0.323984,1,0.1,"{'C': 1, 'gamma': 0.1}",0.452571,0.444571,0.461571,0.452905,0.006944,1
1,57.004790,0.352374,17.299407,0.278106,1,5,"{'C': 1, 'gamma': 5}",0.452571,0.444571,0.461571,0.452905,0.006944,1
2,52.689897,0.558668,14.190683,0.209660,5,0.1,"{'C': 5, 'gamma': 0.1}",0.452571,0.444571,0.461571,0.452905,0.006944,1
3,52.846177,0.085067,14.559902,0.291514,5,5,"{'C': 5, 'gamma': 5}",0.452571,0.444571,0.461571,0.452905,0.006944,1
4,54.365844,0.453559,14.600392,0.323152,10,0.1,"{'C': 10, 'gamma': 0.1}",0.452571,0.444571,0.461571,0.452905,0.006944,1
5,54.277742,0.180565,14.582673,0.622998,10,5,"{'C': 10, 'gamma': 5}",0.452571,0.444571,0.461571,0.452905,0.006944,1


In [20]:
df[['param_C','param_gamma','mean_test_score']]

,param_C,param_gamma,mean_test_score
0,1,0.1,0.452905
1,1,5,0.452905
2,5,0.1,0.452905
3,5,5,0.452905
4,10,0.1,0.452905
5,10,5,0.452905


### 5.Training the model on these *hypermeters*, recording best test result

The optimal parameters for the model is c = 1 and gamma = 0.1 but a/c to default gamma 0 gave better result so optimal model:


In [27]:
final = make_pipeline(svm.SVC(kernel = 'rbf', C = 1.0))
final.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [28]:
y_pred = final.predict(X_test)

In [29]:
#Classification Report for default SVM classifier
print("Model accuracy score is:", accuracy_score(y_pred,y_test))
print(classification_report(y_test, y_pred, target_names=label))

Model accuracy score is: 0.9474444444444444
              precision    recall  f1-score   support

           B       0.96      0.94      0.95       900
           G       0.88      0.92      0.90       900
           F       0.96      0.95      0.95       900
           I       0.93      0.95      0.94       900
     nothing       0.90      0.88      0.89       900
           E       0.93      0.93      0.93       900
           D       0.97      0.96      0.96       900
           H       0.97      0.97      0.97       900
           C       0.99      0.99      0.99       900
           A       1.00      1.00      1.00       900

    accuracy                           0.95      9000
   macro avg       0.95      0.95      0.95      9000
weighted avg       0.95      0.95      0.95      9000

